In [ ]:
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
import numpy as np
import random

In [ ]:
LFP = Structure.from_file("/data/kevinxhan/dist_chgnet/LiFePO4.cif")

In [ ]:
s = LFP.make_supercell((10, 10, 10), in_place=False)
print("# of atoms in system:", len(s))
print(s.composition)
print(s.lattice)
n_Li = 4000
site_indices_to_remove = random.sample(range(0,n_Li), int(n_Li/2))
s.remove_sites(site_indices_to_remove)
print(s.composition)
print("# of atoms in system:", len(s))
supercell = s.make_supercell([2, 1, 1], in_place=False)
print("# of atoms in system:", len(supercell))
print(supercell.composition)
print(supercell.lattice)
LFP_atoms = AseAtomsAdaptor().get_atoms(supercell)
print(len(LFP_atoms))

In [ ]:
import matgl
from DistMLIP.implementations.matgl import Potential_Dist, PESCalculator_Dist
from DistMLIP.implementations.matgl import CHGNet_Dist, TensorNet_Dist
from DistMLIP.implementations.matgl import MolecularDynamics, Relaxer


model = matgl.load_model('/data/shared/Bowen_data/Kevin/TensorNet-MatPES-r2SCAN-2025.4.22-PES-0.3M').model
dist_model = TensorNet_Dist.from_existing(model).eval()
dist_model.enable_distributed_mode(["cpu", 7])


In [ ]:
pes = Potential_Dist(model=dist_model, num_threads=128)

In [ ]:
model.linear_dist.weight.device

In [ ]:
driver = MolecularDynamics(
    LFP_atoms,
    potential=pes,
    timestep=2,
    temperature=1000,
    # logfile="H2O_0.1M_1000k.log",
    # trajectory="H2O_0.1M_1000k.traj",
    loginterval=100000
)

driver.run(2000)

In [ ]:
import re
import matplotlib.pyplot as plt

# Read file content into a string
with open('nohup.out', 'r') as file:
    data = file.read()

# Regex patterns
allocated_pattern = re.compile(r"Allocated \(GB\): \[([^\]]+)\]")
reserved_pattern = re.compile(r"Reserved\s+\(GB\): \[([^\]]+)\]")
distributed_pattern = re.compile(r"Distributed object creation: ([\d.]+)")
step_time_pattern = re.compile(r"Step time: ([\d.]+)")

# Extract values
allocated_matches = allocated_pattern.findall(data)
reserved_matches = reserved_pattern.findall(data)
distributed_times = [float(x) for x in distributed_pattern.findall(data)]
step_times = [3] + [float(x) for x in step_time_pattern.findall(data)]

# Compute averages
average_allocated = []
average_reserved = []

for alloc, resv in zip(allocated_matches, reserved_matches):
    alloc_vals = [float(val.strip().strip("'")) for val in alloc.split(',')]
    resv_vals = [float(val.strip().strip("'")) for val in resv.split(',')]
    average_allocated.append(sum(alloc_vals) / len(alloc_vals))
    average_reserved.append(sum(resv_vals) / len(resv_vals))

# Time steps
steps = list(range(len(average_allocated)))

# Plot 1: Memory usage
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(steps, average_allocated, label='Avg Allocated Memory', linewidth=2)
plt.plot(steps, average_reserved, label='Avg Reserved Memory', linewidth=2)
plt.title('Memory Usage per Step')
plt.xlabel('Step')
plt.ylabel('Memory (GB)')
plt.legend()
plt.grid(True)

# Plot 2: Timing info
plt.subplot(1, 2, 2)
plt.plot(steps, distributed_times, label='Distributed Object Creation Time', linewidth=2)
plt.plot(steps, step_times, label='Total Step Time', linestyle='--', linewidth=2)
plt.title('Timing Information per Step')
plt.xlabel('Step')
plt.ylabel('Time (s)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()
